In [1]:
import numpy as np
import pandas as pd

## Loading the Data - companies.txt and round2.csv

The data is loaded using the encoding ISO-8859-1. Since the data is in different encodings,
the special characters are also handled. The key columns have been cased to lowercase to support 
case-insensitive matching while merging the dataframes.

In [2]:
companies = pd.read_csv("companies.txt", sep = '\t', encoding = "ISO-8859-1")
companies.permalink = companies.permalink.str.encode('utf-8').str.decode('ascii', 'ignore')
companies.permalink = companies.permalink.str.lower()
print(companies.shape)
companies.head()

(66368, 10)


,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at
0,/organization/-fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN
1,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014
2,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN
3,/organization/0-6-com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007
4,/organization/004-technologies,004 Technologies,http://004gmbh.de/en/004-interact,Software,operating,USA,IL,"Springfield, Illinois",Champaign,01-01-2010


In [3]:
rounds2 = pd.read_csv("rounds2.csv", encoding = "ISO-8859-1")
rounds2.company_permalink = rounds2.company_permalink.str.encode('utf-8').str.decode('ascii', 'ignore')
rounds2.company_permalink = rounds2.company_permalink.str.lower();
print(rounds2.shape)
rounds2.head()

(114949, 6)


,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0
1,/organization/-qounter,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN
2,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,01-03-2014,700000.0
3,/organization/-the-one-of-them-inc-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0
4,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0


### Basic Analysis

In [4]:
# How many unique companies are present in companies?
companies.groupby('permalink').nunique().count()

permalink        66368
name             66368
homepage_url     66368
category_list    66368
status           66368
country_code     66368
state_code       66368
region           66368
city             66368
founded_at       66368
dtype: int64

In [5]:
# How many unique companies are present in rounds2?
rounds2.groupby('company_permalink').nunique().count()

company_permalink          66368
funding_round_permalink    66368
funding_round_type         66368
funding_round_code         66368
funded_at                  66368
raised_amount_usd          66368
dtype: int64

In [6]:
# In the companies data frame, which column can be used as the unique key for each company? Write the name of the column.
companies.info()
# permalink and name can be used to identify a company. But since name has some null values, 
#'permalink' must be used as unique key.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66368 entries, 0 to 66367
Data columns (total 10 columns):
permalink        66368 non-null object
name             66367 non-null object
homepage_url     61310 non-null object
category_list    63220 non-null object
status           66368 non-null object
country_code     59410 non-null object
state_code       57821 non-null object
region           58338 non-null object
city             58340 non-null object
founded_at       51147 non-null object
dtypes: object(10)
memory usage: 5.1+ MB


In [7]:
# Are there any companies in the rounds2 file which are not present in companies? Answer yes or no: Y/N
rounds2[~(rounds2['company_permalink'].isin(companies['permalink']))].reset_index(drop=True)
# No

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd


### Cleaning the Data

In [8]:
companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66368 entries, 0 to 66367
Data columns (total 10 columns):
permalink        66368 non-null object
name             66367 non-null object
homepage_url     61310 non-null object
category_list    63220 non-null object
status           66368 non-null object
country_code     59410 non-null object
state_code       57821 non-null object
region           58338 non-null object
city             58340 non-null object
founded_at       51147 non-null object
dtypes: object(10)
memory usage: 5.1+ MB


In [9]:
rounds2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114949 entries, 0 to 114948
Data columns (total 6 columns):
company_permalink          114949 non-null object
funding_round_permalink    114949 non-null object
funding_round_type         114949 non-null object
funding_round_code         31140 non-null object
funded_at                  114949 non-null object
raised_amount_usd          94959 non-null float64
dtypes: float64(1), object(5)
memory usage: 5.3+ MB


#### Identifying the missing values in companies data and rounds2 data.

In [10]:
round(companies.isnull().sum()/len(companies.index) * 100, 2)

permalink         0.00
name              0.00
homepage_url      7.62
category_list     4.74
status            0.00
country_code     10.48
state_code       12.88
region           12.10
city             12.10
founded_at       22.93
dtype: float64

In [11]:
round(rounds2.isnull().sum()/len(rounds2.index) * 100, 2)

company_permalink           0.00
funding_round_permalink     0.00
funding_round_type          0.00
funding_round_code         72.91
funded_at                   0.00
raised_amount_usd          17.39
dtype: float64

In companies data, homepage_url, state_code, region, city, founded_at and in rounds2.csv, funding_round_code column has more than 15% missing values and these columns are not invovled in our analysis so we can drop them.

In [12]:
companies.drop(["homepage_url", "state_code", "region", "city", "founded_at"], axis = 1, inplace = True)
rounds2.drop("funding_round_code", axis = 1, inplace = True)

In [13]:
round(companies.isnull().sum()/len(companies.index) * 100, 2)

permalink         0.00
name              0.00
category_list     4.74
status            0.00
country_code     10.48
dtype: float64

In [14]:
round(rounds2.isnull().sum()/len(rounds2.index) * 100, 2)

company_permalink           0.00
funding_round_permalink     0.00
funding_round_type          0.00
funded_at                   0.00
raised_amount_usd          17.39
dtype: float64

Lets remove the missing values from country_code in companies.csv and raised_amount_usd in rounds2. Since, the mapping for 

In [15]:
companies.dropna(subset = ["category_list", "country_code"], inplace = True)
rounds2.dropna(subset = ["raised_amount_usd"], inplace = True)

In [16]:
round(companies.isnull().sum()/len(companies.index) * 100, 2)

permalink        0.0
name             0.0
category_list    0.0
status           0.0
country_code     0.0
dtype: float64

In [17]:
round(rounds2.isnull().sum()/len(rounds2.index) * 100, 2)

company_permalink          0.0
funding_round_permalink    0.0
funding_round_type         0.0
funded_at                  0.0
raised_amount_usd          0.0
dtype: float64

The missing data has been handled. 

### Merging the data

Merge the two data frames so that all variables (columns) in the companies frame are added to the rounds2 data frame. 
Name the merged frame master_frame. 
Lets merge the dataframes on the keys - company_permalink from rounds2 and permalink from companies.
Lets first check if the join columns are of same datatype.

In [18]:
print(rounds2['company_permalink'].dtypes)
print(companies['permalink'].dtypes)

object
object


In [19]:
# How many observations are present in master_frame?
master_frame = pd.merge(rounds2, companies, left_on = 'company_permalink', right_on ='permalink', how = 'left')
print(master_frame.shape)
master_frame.head(5)

(94959, 10)


,company_permalink,funding_round_permalink,funding_round_type,funded_at,raised_amount_usd,permalink,name,category_list,status,country_code
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,05-01-2015,10000000.0,/organization/-fame,#fame,Media,operating,IND
1,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,01-03-2014,700000.0,/organization/-qounter,:Qounter,Application Platforms|Real Time|Social Network...,operating,USA
2,/organization/-the-one-of-them-inc-,/funding-round/650b8f704416801069bb178a1418776b,venture,30-01-2014,3406878.0,NaN,NaN,NaN,NaN,NaN
3,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,19-03-2008,2000000.0,/organization/0-6-com,0-6.com,Curated Web,operating,CHN
4,/organization/01games-technology,/funding-round/7d53696f2b4f607a2f2a8cbb83d01839,undisclosed,01-07-2014,41250.0,/organization/01games-technology,01Games Technology,Games,operating,HKG


#### Finding the primary_sector and map it with the main_sector for sector type analysis

In [20]:
# finding the primary sector from category list for venture funding types
master_frame['primary_sector'] = master_frame['category_list'].str.split("|", expand = True)[0]
master_frame.head()

,company_permalink,funding_round_permalink,funding_round_type,funded_at,raised_amount_usd,permalink,name,category_list,status,country_code,primary_sector
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,05-01-2015,10000000.0,/organization/-fame,#fame,Media,operating,IND,Media
1,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,01-03-2014,700000.0,/organization/-qounter,:Qounter,Application Platforms|Real Time|Social Network...,operating,USA,Application Platforms
2,/organization/-the-one-of-them-inc-,/funding-round/650b8f704416801069bb178a1418776b,venture,30-01-2014,3406878.0,NaN,NaN,NaN,NaN,NaN,NaN
3,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,19-03-2008,2000000.0,/organization/0-6-com,0-6.com,Curated Web,operating,CHN,Curated Web
4,/organization/01games-technology,/funding-round/7d53696f2b4f607a2f2a8cbb83d01839,undisclosed,01-07-2014,41250.0,/organization/01games-technology,01Games Technology,Games,operating,HKG,Games


In [21]:
# loading the mapping data
mapping = pd.read_csv("mapping.csv")
print(mapping.shape)
mapping.head()

(688, 10)


,category_list,Automotive & Sports,Blanks,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising"
0,NaN,0,1,0,0,0,0,0,0,0
1,3D,0,0,0,0,0,1,0,0,0
2,3D Printing,0,0,0,0,0,1,0,0,0
3,3D Technology,0,0,0,0,0,1,0,0,0
4,Accounting,0,0,0,0,0,0,0,0,1


In [22]:
# cleaning the mapping data
mapping.category_list.values

array([nan, '3D', '3D Printing', '3D Technology', 'Accounting',
       'Active Lifestyle', 'Ad Targeting', 'Advanced Materials',
       'Adventure Travel', 'Advertising', 'Advertising Exchanges',
       'Advertising Networks', 'Advertising Platforms', 'Advice',
       'Aerospace', 'Agriculture', 'Air Pollution Control', 'Algorithms',
       'All Markets', 'All Students', 'Alter0tive Medicine', 'Alumni',
       'A0lytics', 'Android', 'Angels', 'Animal Feed',
       'Anything Capital Intensive', 'App Discovery', 'App Marketing',
       'App Stores', 'Application Performance Monitoring',
       'Application Platforms', 'Apps', 'Aquaculture', 'Architecture',
       'Archiving', 'Art', 'Artificial Intelligence', 'Artists Globally',
       'Assisitive Technology', 'Assisted Living', 'Auctions', 'Audio',
       'Audiobooks', 'Augmented Reality', 'Auto', 'Automated Kiosk',
       'Automotive', 'B2B', 'B2B Express Delivery', 'Babies',
       'Baby Accessories', 'Baby Boomers', 'Baby Safety', 'B

As we can observe, some categories names are incorrect. and on analysis it is found that '0' is placed instead of 'na' at variuos places.

In [23]:
mapping.category_list = mapping.category_list.str.replace("0", "na")

The data is clean now, But the data is in wide format. It is preferable to convert this data into long format, to ease the mapping of primary sector to mian sector.

In [24]:
mapping = pd.melt(mapping, id_vars='category_list', value_vars = mapping.columns[1:])
mapping = mapping[mapping.value != 0]
mapping.head()

,category_list,variable,value
8,Adventure Travel,Automotive & Sports,1
14,Aerospace,Automotive & Sports,1
45,Auto,Automotive & Sports,1
46,Automated Kiosk,Automotive & Sports,1
47,Automotive,Automotive & Sports,1


In [25]:
# Renaming the columns and dropping unnecessary columns before merging the mapping data
mapping.drop("value", axis = 1, inplace = True)
mapping.rename( columns={"category_list": "category", "variable" : "main_sector"} , inplace = True)
mapping.head()

,category,main_sector
8,Adventure Travel,Automotive & Sports
14,Aerospace,Automotive & Sports
45,Auto,Automotive & Sports
46,Automated Kiosk,Automotive & Sports
47,Automotive,Automotive & Sports


Merge the master_frame_venture with mapping data to get the main sector.

In [26]:
master_data = pd.merge(master_frame, mapping, how = "left", 
                                       left_on = "primary_sector", right_on = "category")
master_data.head()

,company_permalink,funding_round_permalink,funding_round_type,funded_at,raised_amount_usd,permalink,name,category_list,status,country_code,primary_sector,category,main_sector
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,05-01-2015,10000000.0,/organization/-fame,#fame,Media,operating,IND,Media,Media,Entertainment
1,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,01-03-2014,700000.0,/organization/-qounter,:Qounter,Application Platforms|Real Time|Social Network...,operating,USA,Application Platforms,Application Platforms,"News, Search and Messaging"
2,/organization/-the-one-of-them-inc-,/funding-round/650b8f704416801069bb178a1418776b,venture,30-01-2014,3406878.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Blanks
3,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,19-03-2008,2000000.0,/organization/0-6-com,0-6.com,Curated Web,operating,CHN,Curated Web,Curated Web,"News, Search and Messaging"
4,/organization/01games-technology,/funding-round/7d53696f2b4f607a2f2a8cbb83d01839,undisclosed,01-07-2014,41250.0,/organization/01games-technology,01Games Technology,Games,operating,HKG,Games,Games,Entertainment


In [49]:
# exporting data for data visualisation on tableau
master_data.to_csv("investment_case_study_analysis.csv")

### Investment Type Analysis

In [28]:
# Calculate the average investment amount for each of the four funding types (venture, angel, seed, and private equity)
pd.DataFrame(master_data[master_data.funding_round_type.isin(["venture", "angel", "seed", "private_equity"])]
             .groupby("funding_round_type")["raised_amount_usd"].mean()/1000000)
# the values are in million usd for ease of reading

,raised_amount_usd
funding_round_type,
angel,0.958694
private_equity,73.308593
seed,0.719818
venture,11.748949


From investment type analysis, we can observe that the suitable investment type for SparksFunds (invest between 5 to 15 million USD per investment round) is: **Venture**

### Country Type Analysis

In [29]:
# Spark Funds wants to see the top nine countries which have received the highest total funding 
# (across ALL sectors for the chosen investment type)
# For the chosen investment type, make a data frame named top9 with the top nine countries 
master_data_venture = pd.DataFrame(master_data.loc[(master_data.funding_round_type == 'venture'),:])
top9 = pd.DataFrame(master_data_venture.groupby("country_code")["raised_amount_usd"].sum().nlargest(9)/1000000)
top9

,raised_amount_usd
country_code,
USA,420068.029342
CHN,39338.918773
GBR,20072.813004
IND,14261.508718
CAN,9482.217668
FRA,7226.851352
ISR,6854.350477
DEU,6306.921981
JPN,3167.647127


Using this analysis and the list of English-speaking countries provided, the top three English-speaking countries are:  **USA, GBR, IND**

### Sector Type Analysis

Create three separate data frames D1, D2 and D3 for each of the three countries containing the observations of funding type FT falling within the 5-15 million USD range. The three data frames should contain:

All the columns of the master_frame along with the primary sector and the main sector

The total number (or count) of investments for each main sector in a separate column

The total amount invested in each main sector in a separate column

In [32]:
# D1 for USA
master_data_usa = master_data[(master_data.country_code == 'USA')
                              & (master_data.funding_round_type == 'venture')
                              & (master_data.raised_amount_usd >= 5000000)
                              & (master_data.raised_amount_usd <= 15000000)]
D1 = master_data_usa.copy()
D1["total_investments_sector_wise"] = D1.groupby(["country_code", "main_sector"])["raised_amount_usd"].transform('count')
D1["total_amount_invested_sector_wise"] = D1.groupby(["country_code", "main_sector"])["raised_amount_usd"].transform('sum')
D1.head()

,company_permalink,funding_round_permalink,funding_round_type,funded_at,raised_amount_usd,permalink,name,category_list,status,country_code,primary_sector,category,main_sector,total_investments_sector_wise,total_amount_invested_sector_wise
10,/organization/0xdata,/funding-round/e1cfcbe1bdf4c70277c5f29a3482f24e,venture,19-07-2014,8900000.0,/organization/0xdata,H2O.ai,Analytics,operating,USA,Analytics,Analytics,"Social, Finance, Analytics, Advertising",2714.0,2.380738e+10
16,/organization/1-mainstream,/funding-round/b952cbaf401f310927430c97b68162ea,venture,17-03-2015,5000000.0,/organization/1-mainstream,1 Mainstream,Apps|Cable|Distribution|Software,acquired,USA,Apps,Apps,"News, Search and Messaging",1582.0,1.395957e+10
78,/organization/128-technology,/funding-round/fb6216a30cb566ede89e0bee0623a634,venture,16-12-2014,11999347.0,/organization/128-technology,128 Technology,Service Providers|Technology,operating,USA,Service Providers,Service Providers,Others,2950.0,2.632101e+10
84,/organization/1366-technologies,/funding-round/424129ce1235cfab2655ee81305f7c2b,venture,15-10-2013,15000000.0,/organization/1366-technologies,1366 Technologies,Manufacturing,operating,USA,Manufacturing,Manufacturing,Manufacturing,799.0,7.258553e+09
85,/organization/1366-technologies,/funding-round/6d3f3797371956ece035b8478c1441b2,venture,09-04-2015,5000000.0,/organization/1366-technologies,1366 Technologies,Manufacturing,operating,USA,Manufacturing,Manufacturing,Manufacturing,799.0,7.258553e+09


In [33]:
# Count and Sum of Investments Sector wise for USA
pd.DataFrame(D1.groupby(["country_code", "main_sector"])["raised_amount_usd"].agg({'count', 'sum'})).sort_values('count',ascending=False)

sum  count
country_code main_sector                                                 
USA          Others                                   2.632101e+10   2950
             Social, Finance, Analytics, Advertising  2.380738e+10   2714
             Cleantech / Semiconductors               2.120663e+10   2300
             News, Search and Messaging               1.395957e+10   1582
             Health                                   8.211859e+09    909
             Manufacturing                            7.258553e+09    799
             Entertainment                            5.099198e+09    591
             Automotive & Sports                      1.454104e+09    167

In [34]:
# Total Count and Sum of Investments for USA
print(D1["raised_amount_usd"].count())
print(D1["raised_amount_usd"].sum())

12064
107766584223.0


In [35]:
# For point 3 (top sector count-wise), which company received the highest investment for USA
D1.set_index("company_permalink", inplace = True)
pd.DataFrame(D1[D1.main_sector == "Others"].raised_amount_usd.nlargest(1))

,raised_amount_usd
company_permalink,
/organization/1stdibs,15000000.0


In [36]:
# For point 4 (second best sector count-wise), which company received the highest investment for USA
pd.DataFrame(D1[D1.main_sector == "Social, Finance, Analytics, Advertising"].raised_amount_usd.nlargest(1))

,raised_amount_usd
company_permalink,
/organization/admeld,15000000.0


In [38]:
# D2 for GBR
master_data_gbr = master_data[(master_data.country_code == 'GBR')  
                                        & (master_data.funding_round_type == 'venture')
                                        & (master_data.raised_amount_usd >= 5000000)
                                        & (master_data.raised_amount_usd <= 15000000)]
D2 = master_data_gbr.copy()
D2["total_investments_sector_wise"] = D2.groupby(["country_code", "main_sector"])["raised_amount_usd"].transform('count')
D2["total_amount_invested_sector_wise"] = D2.groupby(["country_code", "main_sector"])["raised_amount_usd"].transform('sum')
D2.head()

,company_permalink,funding_round_permalink,funding_round_type,funded_at,raised_amount_usd,permalink,name,category_list,status,country_code,primary_sector,category,main_sector,total_investments_sector_wise,total_amount_invested_sector_wise
309,/organization/365scores,/funding-round/48212f931f542fdef78810bc87aef086,venture,29-09-2014,5500000.0,/organization/365scores,365Scores,Android|Apps|iPhone|Mobile|Sports,operating,GBR,Android,Android,"Social, Finance, Analytics, Advertising",133.0,1.089404e+09
671,/organization/7digital,/funding-round/b5ad7ed7baddd3974bd51403f17dd88f,venture,01-01-2008,8468328.0,/organization/7digital,7digital,Content Creators|Content Delivery|Licensing|Mu...,acquired,GBR,Content Creators,Content Creators,Entertainment,56.0,4.827847e+08
672,/organization/7digital,/funding-round/eafacfcceb1fbc4fd605f641b603313e,venture,19-10-2012,10000000.0,/organization/7digital,7digital,Content Creators|Content Delivery|Licensing|Mu...,acquired,GBR,Content Creators,Content Creators,Entertainment,56.0,4.827847e+08
730,/organization/90min,/funding-round/21a2cbf6f2fb2a1c2a61e04bf930dfe6,venture,06-10-2015,15000000.0,/organization/90min,90min,Media|News|Publishing|Soccer|Sports,operating,GBR,Media,Media,Entertainment,56.0,4.827847e+08
731,/organization/90min,/funding-round/bd626ed022f5c66574b1afe234f3c90d,venture,07-05-2013,5800000.0,/organization/90min,90min,Media|News|Publishing|Soccer|Sports,operating,GBR,Media,Media,Entertainment,56.0,4.827847e+08


In [39]:
# Count and Sum of Investments Sector wise for GBR
pd.DataFrame(D2.groupby(["country_code", "main_sector"])["raised_amount_usd"].agg({'count', 'sum'})).sort_values('count',ascending=False)

sum  count
country_code main_sector                                                 
GBR          Others                                   1.283624e+09    147
             Social, Finance, Analytics, Advertising  1.089404e+09    133
             Cleantech / Semiconductors               1.150140e+09    128
             News, Search and Messaging               6.157462e+08     73
             Entertainment                            4.827847e+08     56
             Manufacturing                            3.619403e+08     42
             Health                                   2.145375e+08     24
             Automotive & Sports                      1.670516e+08     16

In [40]:
# Total Count and Sum of Investments for GBR
print(D2["raised_amount_usd"].count())
print(D2["raised_amount_usd"].sum())

621
5379078691.0


In [41]:
# For point 3 (top sector count-wise), which company received the highest investment for GBR
D2.set_index("company_permalink", inplace = True)
pd.DataFrame(D2[D2.main_sector == "Others"].raised_amount_usd.nlargest(1))

,raised_amount_usd
company_permalink,
/organization/azure-solutions,15000000.0


In [42]:
# For point 4 (second best sector count-wise), which company received the highest investment for GBR
pd.DataFrame(D2[D2.main_sector == "Social, Finance, Analytics, Advertising"].raised_amount_usd.nlargest(1))

,raised_amount_usd
company_permalink,
/organization/mythings,15000000.0


In [43]:
# D3 for IND
master_data_ind = master_data[(master_data.country_code == 'IND') 
                                        & (master_data.funding_round_type == 'venture')
                                        & (master_data.raised_amount_usd >= 5000000)
                                        & (master_data.raised_amount_usd <= 15000000)]
D3 = master_data_ind.copy()
D3["total_investments_sector_wise"] = D3.groupby(["country_code", "main_sector"])["raised_amount_usd"].transform('count')
D3["total_amount_invested_sector_wise"] = D3.groupby(["country_code", "main_sector"])["raised_amount_usd"].transform('sum')
D3.head()

,company_permalink,funding_round_permalink,funding_round_type,funded_at,raised_amount_usd,permalink,name,category_list,status,country_code,primary_sector,category,main_sector,total_investments_sector_wise,total_amount_invested_sector_wise
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,05-01-2015,10000000.0,/organization/-fame,#fame,Media,operating,IND,Media,Media,Entertainment,33,2.808300e+08
178,/organization/21diamonds-india,/funding-round/6de7ffef8091ba9f33821f4b861f434a,venture,15-11-2012,6369507.0,/organization/21diamonds-india,21Diamonds,E-Commerce,operating,IND,E-Commerce,E-Commerce,Others,110,1.013410e+09
810,/organization/a-little-world,/funding-round/18d98f82ed392b1609975b81f3e8b3fb,venture,09-09-2008,6410000.0,/organization/a-little-world,A LITTLE WORLD,Finance,operating,IND,Finance,Finance,"Social, Finance, Analytics, Advertising",60,5.505496e+08
2051,/organization/adlabs-imagica,/funding-round/508d3c83daaae9fda3ba6f9682c78f6c,venture,28-10-2014,8180000.0,/organization/adlabs-imagica,Adlabs Imagica,Entertainment|Tourism,operating,IND,Entertainment,Entertainment,Entertainment,33,2.808300e+08
2849,/organization/agile,/funding-round/cd3dd1c98ce9d0f632d8752163941674,venture,01-05-2011,5740000.0,/organization/agile,Agile,Finance|Finance Technology|FinTech|Insurance,operating,IND,Finance,Finance,"Social, Finance, Analytics, Advertising",60,5.505496e+08


In [44]:
# Count and Sum of Investments Sector wise for IND
pd.DataFrame(D3.groupby(["country_code", "main_sector"])["raised_amount_usd"].agg({'count', 'sum'})).sort_values('count',ascending=False)

sum  count
country_code main_sector                                                 
IND          Others                                   1.013410e+09    110
             Social, Finance, Analytics, Advertising  5.505496e+08     60
             News, Search and Messaging               4.338345e+08     52
             Entertainment                            2.808300e+08     33
             Manufacturing                            2.009000e+08     21
             Cleantech / Semiconductors               1.653800e+08     20
             Health                                   1.677400e+08     19
             Automotive & Sports                      1.369000e+08     13

In [45]:
# Total Count and Sum of Investments for IND
print(D3["raised_amount_usd"].count())
print(D3["raised_amount_usd"].sum())

328
2949543602.0


In [46]:
# For point 3 (top sector count-wise), which company received the highest investment for IND
D3.set_index("company_permalink", inplace = True)
pd.DataFrame(D3[D3.main_sector == "Others"].raised_amount_usd.nlargest(1))

,raised_amount_usd
company_permalink,
/organization/caratlane,15000000.0


In [47]:
# For point 4 (second best sector count-wise), which company received the highest investment for IND
pd.DataFrame(D3[D3.main_sector == "Social, Finance, Analytics, Advertising"].raised_amount_usd.nlargest(1))

,raised_amount_usd
company_permalink,
/organization/apnaloan,15000000.0
